# Thyroid Disease EDA, Classification and Ensembling

Predicting from the MLDataR package

The purpose of this notebook is to build my Data Sciece and Machine Learning skills. This is my third complete notebook. You can check other notebooks at my [github repository](https://github.com/elijahrona/Elijah-Rona-ML-Journey). Follow my everyday ML journey on [Twitter](https://twitter.com/elijah_rona).

## Loading Libraries

In [ ]:
library(MLDataR) #Source of dataset
library(tidyverse) #EDA plus others
library(tidymodels) #Machine Leaning
library(skimr) #Data exploration
library(stacks) #Ensembling
library(ggpubr) #Plot arrangements

## Loading and Exploring the Dataset

In [ ]:
df <- thyroid_disease #from the library

#Let us explore the dataset
skimmed <- skim(df)
head(skimmed)

In [ ]:
df <- df[complete.cases(df),]
dim(df)
head(df)

The data looks okay and is ready for EDA and Modeling

## Exploratory Data Analysis

Let us create a loop fuction to create several plots to help understand our data better

In [ ]:
#Assign the variables
predicator <- names(df)[2:27]
predicator <- predicator[-c(1, 18, 20, 22, 24, 26)]
predicator <- set_names(predicator)
response <- names(df)[1]
response <- set_names(response)

#create loop function
chart_fun <- function(x, y) {
  ggplot(df, aes(fill=.data[[y]], x=as.factor(.data[[x]]))) + 
    geom_bar(position="dodge", stat="count", colour="black") +
    theme(
      panel.background = element_rect(fill = "white",
                                      colour = "white",
                                      size = 0.5, linetype = "solid"))
}

#plot charts
finally <- map(response, ~map(predicator, chart_fun, y = .x))

Let us test the function if it works with the "Pregnant" column

In [ ]:
library(repr) ;
options(repr.plot.width=16, repr.plot.height = 9)
finally$ThryroidClass$pregnant

It works!! Time to combine all plots

In [ ]:
cowplot::plot_grid(plotlist = finally[[1]], scale = 1)

Explaining these plots can be quite difficult, so let us use statistical EDA through which we can know if each column is effective in predicting the response column or not.

Note that for the plots above, we only used columns with 1s and 0s (Just like the response column). The best form of stat EDA for such columns is chi-square. Note that if the P-Value is lower than 0.05, we will accept that the column is a good predicator.

In [ ]:
chi_square_results <- purrr::map(df[,predicator[-c(16:20)]], ~chisq.test(.x, df$ThryroidClass))
chi_square_results

There are many columns with P-Value lower than 0.05. This means that they are effective in predicting the response column. We, however, will use every column.

Time to do EDA for the other normal continous columns

In [ ]:
#get columns by index (You can call columns with this...)
predicator1 <- names(df)[2:27]
predicator1 <- predicator1[c(1, 18, 20, 22, 24, 26)]
predicator1 <- set_names(predicator1)

chart_fun1 <- function(x, y) {
  ggplot(df, aes(y=.data[[x]], x=as.factor(.data[[y]]))) + 
    geom_boxplot(fill = 'bisque', color = 'black', alpha = 0.3) +
    geom_jitter(aes(color = 'blue'), alpha = 0.2) +
    guides(color = "none") +
    theme_minimal() +
    coord_cartesian(ylim = quantile(.data[[x]], c(0, 0.999)))
}

finally1 <- map(response, ~map(predicator1, chart_fun1, y = .x))

cowplot::plot_grid(plotlist = finally1[[1]])

We can see from the boxplots that older people are more likely to have thyroid. To further understand all columns, let us use ANOVA. Same principle of P-Value applies here

In [ ]:
anova_results <- purrr::map(df[,predicator1], ~summary(aov(.x ~ df$ThryroidClass)))
anova_results

Now that we know which column is a good predicator and which is not, it is time to build our model

## Building Out Models
### 1. Splitting Our Dataset

In [ ]:
#Make sure that the response is a factor
df <- df %>%
  mutate(ThryroidClass = as.factor(ThryroidClass))

#split the dataset
set.seed(123)
td_split  <- initial_split(df, 
                             strata = ThryroidClass,
                             breaks = 4)
td_train  <- training(td_split)
td_test   <- testing(td_split)

### 2. Making a Recipe

In [ ]:
td_recipe <- recipe(ThryroidClass ~ ., data = td_train) %>% #Set the formula
  remove_role(ref_src, old_role = "predictor") %>% #We do not want to use the "ref_src" column
  step_normalize(all_numeric_predictors()) %>% #Normalize every numeric predicator
  step_zv(all_predictors()) #Remove predicators with only one value (they are useless)

Let us test the recipe

In [ ]:
rec_sam <- td_recipe %>% 
  prep(training = td_train, retain = TRUE) %>% #so_train is train data
  juice()

head(rec_sam)

The dataset above is exactly what out recipe produced. This is what we will train our model with

### 3 (A). Training a Model (Random Forest-Ranger)

In [ ]:
#Specify the model and mode
td_rf <-
  rand_forest() %>%
  set_engine("ranger") %>% 
  set_mode("classification")

#Make a workflow. We add the recipe we creatd earlier
td_rf_wf <- 
  workflow() %>% 
  add_recipe(td_recipe) %>% 
  add_model(td_rf)

# Fit Model
set.seed(234)
td_rf_fit <- 
  td_rf_wf %>% 
  fit(data = td_train)

Let us predict with our test data

In [ ]:
td_rf_pred <- augment(td_rf_fit, td_test)

### 3 (B). Evaluating Our Model (Random Forest=Ranger)

In [ ]:
td_rf_pred %>%
  roc_auc(truth = ThryroidClass, 
            estimate = .pred_negative)

td_rf_pred %>%
  roc_curve(truth = ThryroidClass, 
            estimate = .pred_negative) %>% 
  autoplot()

The Roc_AUC is wonderful

In [ ]:
caret::confusionMatrix(reference = td_rf_pred$ThryroidClass, data = td_rf_pred$.pred_class, mode='everything', positive='sick')

With an accuracy of 98%, this model is almost perfect. Well, let us try other models

### 4 (A). Building Our Model (XGBOOST)

#We will be tuning this model

In [ ]:
#Get the tuning parameters of a model
args(boost_tree)

In [ ]:
#Specify the model
td_xgboost <- boost_tree(trees = tune(), tree_depth = tune()) %>% 
  set_mode("classification") %>% 
  set_engine("xgboost")

#Set the selected tuning parameters in grid
boost_grid <- grid_regular( #chooses sensible values to try for each hyperparameter
  trees(), tree_depth(),levels = 5)

#Create folds
set.seed(234)
folds <- vfold_cv(td_train)

#Add model and recipe to workflow
td_xgboost_wf <- workflow() %>%
  add_model(td_xgboost) %>%
  add_recipe(td_recipe)

#Start testing different parameters in different grids according to the settings
set.seed(124)
td_xgboost_fold <- td_xgboost_wf %>% 
  tune_grid(resamples = folds, grid = boost_grid)

What is the best hyperparameter combination?

In [ ]:
head(collect_metrics(td_xgboost_fold))

In [ ]:
best_boost <- td_xgboost_fold %>%
  select_best("accuracy")

best_boost

Let us visualize the varirous hyperparameters used

In [ ]:
td_xgboost_fold %>%
  collect_metrics() %>%
  mutate(tree_depth = factor(tree_depth)) %>%
  ggplot(aes(trees, mean, color = tree_depth)) +
  geom_line(size = 1.5, alpha = 0.6) +
  geom_point(size = 2) +
  facet_wrap(~ .metric, scales = "free", nrow = 2) +
  scale_x_log10(labels = scales::label_number()) +
  scale_color_viridis_d(option = "plasma", begin = .9, end = 0)

Since we saved the best hyperparameter combination as "best_boost", we will use that variable in out model

In [ ]:
#Using the best hyperparameters
final_xgboost_wf <- 
  td_xgboost_wf %>% 
  finalize_workflow(best_boost)

#Fitting out model
final_xgboost_fit <- 
  final_xgboost_wf %>%
  last_fit(td_split) #last_fit trains with train data and tests test data automatically

### 4 (B). Evaluating Our Model (XGBOOST)

In [ ]:
collect_metrics(final_xgboost_fit)

Good accuracy

In [ ]:
final_xgboost_fit %>% #You can use this to predict on new data
  collect_predictions() %>% 
  roc_curve(ThryroidClass, .pred_negative) %>% 
  autoplot()

Let us predict with our test dataset

In [ ]:
#Collect predictions
td_xgboost_pred <- final_xgboost_fit %>% #You can use this to predict on new data
  collect_predictions()

#Add the predictions to our testdata set
td_xgboost_pred <- cbind(td_test, td_xgboost_pred)

head(td_xgboost_pred)

In [ ]:
#Confusion matrix
caret::confusionMatrix(reference = td_xgboost_pred$ThryroidClass, data = td_xgboost_pred$.pred_class, mode='everything', positive='sick')

## Ensembling Models

I love ensembling because I love seeing people and machines working together

### 1. Grid and Resample

In [ ]:
ctrl_grid <- stacks::control_stack_grid() #Get the grid function (best for xgboost because we tuned it)
ctrl_res <- stacks::control_stack_resamples() #Get the resamples function (best for random forest because we did not tune it)

metric <- metric_set(accuracy, roc_auc)

### 2. Setting Our Models for Tuning

XGBOOST

In [ ]:
xgboost_res <- 
  tune_grid(
    td_xgboost_wf,
    resamples = folds,
    metrics = metric,
    grid = 4,
    control = ctrl_grid
  )

Random Forest

In [ ]:
rf_res <- 
  fit_resamples(
    td_rf_wf, #workflow
    resamples = folds, #cvfold
    metrics = metric,
    control = ctrl_res
  )

### 3. Time to Add Our Models

In [ ]:
set.seed(234)
model_data_st <-  stacks() %>%
  add_candidates(xgboost_res) %>%
  add_candidates(rf_res)

head(model_data_st)

### 4. Blending Our Models' Predictions

In [ ]:
#Blend the predictions
set.seed(148)
fitted_model_st <-
  model_data_st %>%
  blend_predictions()

Let us visualize the fitted models

In [ ]:
autoplot(fitted_model_st)

Our ensembled model will select the best penalty and other parameters


### 5. Ensembling the Models

In [ ]:
#Ensemble models
set.seed(111)
fitted_model_st <-
  fitted_model_st %>%
  fit_members()

Time to predict with the test data

In [ ]:
#predict test data
test_predict_data <- 
  td_test %>%
  bind_cols(predict(fitted_model_st, .))

In [ ]:
head(test_predict_data)

### 6. Comparing our Models

In [ ]:
#Compare ensembled with members
member_preds <- 
  test_predict_data %>%
  select(ThryroidClass) %>%
  bind_cols(predict(fitted_model_st, td_test, members = TRUE))

map_dfr(member_preds, accuracy, truth = ThryroidClass, data = member_preds) %>%
  mutate(member = colnames(member_preds))

They all seem good

## Confusion Matrix

It elps to visualize our predicted data (in relation to the truth)

In [ ]:
p1 <- conf_mat(td_rf_pred, truth = ThryroidClass, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "Random Forest Alone")

td_xgboost_pred <- td_xgboost_pred[,-1]
p2 <- conf_mat(td_xgboost_pred, truth = ThryroidClass, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "XGBOOST Alone")

p3 <- conf_mat(member_preds, truth = ThryroidClass, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "Ensembled")

p4 <- conf_mat(member_preds, truth = ThryroidClass, estimate = .pred_class_xgboost_res_1_2) %>% 
  autoplot(type = "heatmap") +
  labs(title = "XGBOOST in Ensembled")

p5 <- conf_mat(member_preds, truth = ThryroidClass, estimate = .pred_class_rf_res_1_1) %>% 
  autoplot(type = "heatmap") +
  labs(title = "Random Forest in Ensembled")

ggarrange(p1,p2,p3,p4,p5,
          ncol = 3,
          nrow = 2)

Looking at the matrix, do you thik that our models inproved after the ensembling? Share your thoughts.

## Thank You

Remember that you can link with me on [Twitter](https://twitter.com/elijah_rona).